In [1]:
!workon cv
!pip install requests

'workon' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
###
# search_words = ["MR16"]
img_dir = "./Images/"

In [3]:
import urllib.parse

In [4]:
!pip install selenium
from selenium import webdriver

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [6]:
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = '/Users/HP/Desktop/Scraping/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 81


In [6]:
wd.get('https://google.com')


In [ ]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('MR16')

In [22]:
wd.quit()

In [7]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=3):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(60)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [8]:
import requests

In [9]:
import io
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [10]:
!pip install pillow

In [11]:
import os
import io
from PIL import Image
from io import StringIO

In [12]:
def search_and_download(search_term:str,driver_path:str,target_path='./Images',number_images=15):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=1)
        
    for elem in res:
        persist_image(target_folder,elem)

In [13]:
import time
import hashlib

In [14]:
search_term='MR16'
search_and_download(
    search_term=search_term,
    driver_path=DRIVER_PATH
)

Found: 200 search results. Extracting links from 0:200
Found: 15 image links, done!
SUCCESS - saved https://images-na.ssl-images-amazon.com/images/I/61yPLRQfk3L._AC_SL1200_.jpg - as ./Images\mr16\ca2ee3256f.jpg
SUCCESS - saved https://images.homedepot-static.com/productImages/edc29fac-b8f5-412c-a36a-b5fb46aabb62/svn/bulbrite-led-light-bulbs-771328-64_1000.jpg - as ./Images\mr16\ee7eb1076e.jpg
SUCCESS - saved https://cdn11.bigcommerce.com/s-phjrc3z910/images/stencil/original/products/9793/15737/mr16-mother2__69788.1536248791.jpg?c=2 - as ./Images\mr16\d57e31dddf.jpg
SUCCESS - saved https://cdn3.volusion.com/ftvg6.zh3qc/v/vspfiles/photos/LED-6WMR16COB-2.jpg?v-cache=1476689806 - as ./Images\mr16\226d6579d1.jpg
SUCCESS - saved https://www.bulbs.com/Images/productmed/1/33119.jpg - as ./Images\mr16\5e3406f8f5.jpg
SUCCESS - saved https://smhttp-ssl-40934.nexcesscdn.net/media/catalog/product/cache/1/image/600x/9df78eab33525d08d6e5fb8d27136e95/m/r/mr16-6-8xx-25d-dim.jpg - as ./Images\mr16\10e83